In [ ]:
import json
import pandas as pd

parsedData = []

with open('searchResultsAlign.json', 'r') as json_file:
    data = json.load(json_file)


def reformatData(item):
    newItem = item.copy()
    for key in ['lemma', 'strong', 'sourceText', 'targetText']:
        value = item[key]
        count = len(value.split())
        newItem[key + 'Count'] = count

    del newItem['refs']
    del newItem['count']
    return newItem

for i in range(len(data)):
    item = data[i]
    refs = item['refs']
    newItem = reformatData(item)

    for j in range(len(refs)):
        ref = refs[j]
        newItem_ = newItem.copy()
        newItem_['ref'] = ref
        targetsPos = newItem['targetsPos'][j].split()
        positions = []
        for pos in targetsPos:
            positions.append(int(pos))
        newItem_['targetsPos'] = positions

        deltas = []
        for k in range(len(positions) - 1):
            delta = abs(positions[k] - positions[k + 1])
            deltas.append(delta)
        newItem_['deltas'] = deltas
        
        parsedData.append(newItem_)
        
        
with open('parsedSearchResultsAlign.json', 'w') as json_file:
    json.dump(parsedData, json_file)
    
df = pd.DataFrame.from_dict(parsedData)

print(len(parsedData), "rows found")

df

141 rows found
